In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np

In [ ]:
NUM_CLASSES = 15

def create_model():
    """Create a pre-trained model and add a few layers to it.
    
    Returns:
        model (Model): A pre-trained model with a few layers added on top
    """
    base_model = MobileNetV2(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [ ]:
def train_model(model, test_path, validation_path, epochs=1, batch_size=32):
    """Train the model.
    
    Args:
        model (Model): A pre-trained model with a few layers added on top
        test_path (string): Path to the directory that contains the test set
        validation_path (string): Path to the directory that contains the validation set
        epochs (int): Number of epochs to train the model

    Returns:
        model (Model): A trained model
    """

    train_dataset = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_dataset = train_dataset.flow_from_directory(      test_path,
                                                            target_size=(224, 224),
                                                            batch_size=batch_size,
                                                            class_mode='categorical')

    validation_dataset = ImageDataGenerator(preprocessing_function=preprocess_input)

    validation_dataset = validation_dataset.flow_from_directory(validation_path,
                                                            target_size=(224, 224),
                                                            batch_size=batch_size,
                                                            class_mode='categorical')


    # freeze all layers in the base model
    for layer in model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # train the model on the new data for a few epochs
    model.fit_generator(
        train_dataset,
        steps_per_epoch=2000 // batch_size,
        epochs=epochs,
        validation_data=validation_dataset,
        validation_steps=800 // batch_size)

    # unfreeze the last few layers of the base model
    for layer in model.layers[-5:]:
        layer.trainable = True

    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    # we train our model again (this time fine-tuning the top 2 inception blocks
    # alongside the top Dense layers
    model.fit_generator(
        train_dataset,
        steps_per_epoch=2000 // batch_size,
        epochs=epochs,
        validation_data=validation_dataset,
        validation_steps=800 // batch_size)

    # save the model
    model.save('mobilenetv2.h5')
    return model

In [ ]:
def evaluate_model(model, validation_path, batch_size=32):
    """Evaluate the model
    
    Args:
        model (Model): A trained model

    Returns:
        ndarray: A numpy array containing the loss and accuracy of the model
    """
    validation_dataset = ImageDataGenerator(preprocessing_function=preprocess_input)

    validation_dataset = validation_dataset.flow_from_directory(validation_path,
                                                            target_size=(224, 224),
                                                            batch_size=batch_size,
                                                            class_mode='categorical')
                                                            
    return model.evaluate_generator(validation_dataset, steps=800 // batch_size)
